In [1]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.utils.data import DataLoader

from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
import pickle
from PIL import Image

C:\Users\user\AppData\Local\Temp\ipykernel_22572\2258379951.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
!pip install wandb


In [3]:
!wandb login cd60939d994ad74190b61e223efd4acaca58bc11

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\user/.netrc


In [2]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andrew_tep. Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(project="face_or_profile")

In [5]:
wandb.init(config = {
  "learning_rate": 1e-4,
  "epochs": 5,
  "batch_size": 128
})

In [6]:
import random 
SEED = 2345345

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
TRAIN_DIR = Path('train')
TEST_DIR = Path('test')
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
CUDA_LAUNCH_BLOCKING=1
BATCH_SIZE=128

print(DEVICE)

cuda:0


In [8]:
TEST_DIR

WindowsPath('test')

In [9]:
class FaceDataset(Dataset):
    def __init__(self, files, mode):
        super().__init__()
        self.files = sorted(files)
        self.mode = mode
        
        if self.mode not in DATA_MODES:
            print("This mode is not corresct")
            
        self.len_ = len(self.files)
        
        self.label_encoder = LabelEncoder()
        
        if self.mode != "test":
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)
            
            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                    
                    
    def __len__(self):
        return self.len_
    
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
    
    def __getitem__(self, index):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ])
        
        data_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),])
        
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x/255, dtype = 'float32')
        if self.mode == 'train':
            x = data_transforms(x)
        else:
            x = transform(x)
            
        if self.mode == "test":
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)
            

In [10]:
train_val_files = sorted(list(TRAIN_DIR.rglob('*.png')))
test_files = (list(TEST_DIR.glob('*.png')))

from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

In [11]:
test_files

[WindowsPath('test/ 1.png'),
 WindowsPath('test/ 10.png'),
 WindowsPath('test/ 100.png'),
 WindowsPath('test/ 1000.png'),
 WindowsPath('test/ 10000.png'),
 WindowsPath('test/ 10001.png'),
 WindowsPath('test/ 10002.png'),
 WindowsPath('test/ 10003.png'),
 WindowsPath('test/ 10004.png'),
 WindowsPath('test/ 10005.png'),
 WindowsPath('test/ 10006.png'),
 WindowsPath('test/ 10007.png'),
 WindowsPath('test/ 10008.png'),
 WindowsPath('test/ 10009.png'),
 WindowsPath('test/ 1001.png'),
 WindowsPath('test/ 10010.png'),
 WindowsPath('test/ 10011.png'),
 WindowsPath('test/ 10012.png'),
 WindowsPath('test/ 10013.png'),
 WindowsPath('test/ 10014.png'),
 WindowsPath('test/ 10015.png'),
 WindowsPath('test/ 10016.png'),
 WindowsPath('test/ 10017.png'),
 WindowsPath('test/ 10018.png'),
 WindowsPath('test/ 10019.png'),
 WindowsPath('test/ 1002.png'),
 WindowsPath('test/ 10020.png'),
 WindowsPath('test/ 10021.png'),
 WindowsPath('test/ 10022.png'),
 WindowsPath('test/ 10023.png'),
 WindowsPath('test/ 10

In [12]:
sorted(["10","1","2","6"])

['1', '10', '2', '6']

In [13]:
n_classes = len(np.unique(train_val_labels))
n_classes

2

In [14]:
train_labels = [path.parent.name for path in train_files] 
val_labels = [path.parent.name for path in val_files] 

In [15]:
def create_dct_path_labels(train_files, train_labels):
    dct_simpsons = {}
    for label_i in np.unique(train_labels).tolist():
        dct_simpsons[label_i] = []

    for path_i, label_i in zip(train_files, train_labels):
        dct_simpsons[label_i].append(path_i)

    return dct_simpsons

def print_dct(dct_simpsons):
    for key in dct_simpsons:
        print(f"{key}\t{dct_simpsons[key]}")

In [16]:
dct_path_train = create_dct_path_labels(train_files, train_labels)

In [17]:
for person in dct_path_train:
    print(f"{person}\t{len(dct_path_train[person])}")

enface	9142
profile	9029


In [18]:
def create_dct_from_labels(train_val_labels):
    dct_simpsons = {}
    for label_i in np.unique(train_val_labels).tolist():
        dct_simpsons.update({label_i:train_val_labels.count(label_i)})

    return dct_simpsons

new_train_files = []

for person in dct_path_train:
    new_train_files.extend(dct_path_train[person])

new_train_label = [path.parent.name for path in new_train_files]

In [19]:
val_dataset = FaceDataset(val_files, mode='val')
new_train_dataset = FaceDataset(new_train_files, mode='train')

In [20]:
def fit_epoch(model, train_dataloader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_dataloader:
        inputs = inputs.to(DEVICE) #перенос тензоров на видеокарту
        labels = labels.to(DEVICE)
        optimizer.zero_grad() #обнуляем градиенты, чтобы они не накапливались

        outputs = model(inputs) #прогоняем данные из трейнлоадера чеез нашу сеть (модель). интересно, почему не model.forward(inputs)?
        loss = criterion(outputs, labels) #считаем лосс
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1) #все, что больше 1, приводим к 1
        running_loss += loss.item() * inputs.size(0) #непонятно, как работает .size(0)
        running_corrects += torch.sum(preds == labels.data) #если предсказание совпадает с ответом, увеличиваем на 1 running_corrects
        processed_data += inputs.size(0) 
              
    train_loss = running_loss / processed_data #потери на трейне
    train_acc = running_corrects.cpu().numpy() / processed_data #точность на трейне
#     wandb.log({"loss": train_loss})
#     wandb.log({"train_acc": train_acc})
    return train_loss, train_acc

In [21]:
def eval_epoch(model, val_dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_dataloader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    wandb.log({"loss_val": val_loss})
    wandb.log({"val_acc": val_acc})
    return val_loss, val_acc

In [22]:
def train(train_files, val_files, model, epochs, batch_size):
    train_dataloader = DataLoader(new_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_dataloader, criterion, opt)
            wandb.log({"train_loss": train_loss})
            wandb.log({"train_acc": train_acc})
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_dataloader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
    return history

In [23]:
model = models.alexnet(pretrained=True)

In [24]:
for param in model.parameters():
    param.requires_grad = False

num_features = 9216
model.classifier = nn.Linear(num_features, n_classes)
model = model.to(DEVICE)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [44]:
model_resnet = models.resnet34(pretrained=True)

for param in model_resnet.parameters():
    param.require_grad = False

num_features = model_resnet.fc.in_features
model_resnet.classifier = nn.Linear(num_features, n_classes)

if torch.cuda.is_available():
    model_resnet = model_resnet.cuda()
    print('Training with cuda')
    
model_resnet.classifier

Training with cuda


Linear(in_features=512, out_features=2, bias=True)

In [51]:
%%wandb

history = train(new_train_dataset, val_dataset, model=model, epochs=25, batch_size=BATCH_SIZE)

epoch:   0%|          | 0/25 [00:00<?, ?it/s]

loss 0.022508517886814582

Epoch 001 train_loss: 0.0225     val_loss 0.0521 train_acc 0.9954 val_acc 0.9903
loss 0.01573429646794206

Epoch 002 train_loss: 0.0157     val_loss 0.0479 train_acc 0.9961 val_acc 0.9926
loss 0.01828465998065025

Epoch 003 train_loss: 0.0183     val_loss 0.0565 train_acc 0.9959 val_acc 0.9911
loss 0.018388053384578287

Epoch 004 train_loss: 0.0184     val_loss 0.0659 train_acc 0.9961 val_acc 0.9894
loss 0.02047137892855472

Epoch 005 train_loss: 0.0205     val_loss 0.0457 train_acc 0.9960 val_acc 0.9934
loss 0.029618750943535922

Epoch 006 train_loss: 0.0296     val_loss 0.0488 train_acc 0.9949 val_acc 0.9929
loss 0.017351706781718535

Epoch 007 train_loss: 0.0174     val_loss 0.0665 train_acc 0.9960 val_acc 0.9901
loss 0.014246633922634486

Epoch 008 train_loss: 0.0142     val_loss 0.0617 train_acc 0.9970 val_acc 0.9908
loss 0.017539239355534825

Epoch 009 train_loss: 0.0175     val_loss 0.0939 train_acc 0.9968 val_acc 0.9851
loss 0.01339232771324749

Epoch

In [52]:
def predict(model, test_loader):
    with torch.no_grad(): 
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [53]:
def predict_one_sample(model, inputs, device=DEVICE):
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

In [54]:
random_characters = int(np.random.uniform(0,1000))
ex_img, true_label = val_dataset[random_characters]
probs_im = predict_one_sample(model, ex_img.unsqueeze(0))

idxs = list(map(int, np.random.uniform(0,1000, 20)))
imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]

probs_ims = predict(model, imgs)

label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

y_pred = np.argmax(probs_ims,-1)

actual_labels = [val_dataset[id][1] for id in idxs]

preds_class = [label_encoder.classes_[i] for i in y_pred]

In [55]:
test_dataset = FaceDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
probs = predict(model, test_loader) 

preds = label_encoder.inverse_transform(np.argmax(probs, axis=1)) 
test_filenames = [path.name for path in test_dataset.files]

In [56]:
test_filenames

[' 1.png',
 ' 10.png',
 ' 100.png',
 ' 1000.png',
 ' 10000.png',
 ' 10001.png',
 ' 10002.png',
 ' 10003.png',
 ' 10004.png',
 ' 10005.png',
 ' 10006.png',
 ' 10007.png',
 ' 10008.png',
 ' 10009.png',
 ' 1001.png',
 ' 10010.png',
 ' 10011.png',
 ' 10012.png',
 ' 10013.png',
 ' 10014.png',
 ' 10015.png',
 ' 10016.png',
 ' 10017.png',
 ' 10018.png',
 ' 10019.png',
 ' 1002.png',
 ' 10020.png',
 ' 10021.png',
 ' 10022.png',
 ' 10023.png',
 ' 10024.png',
 ' 10025.png',
 ' 10026.png',
 ' 10027.png',
 ' 10028.png',
 ' 10029.png',
 ' 1003.png',
 ' 10030.png',
 ' 10031.png',
 ' 10032.png',
 ' 10033.png',
 ' 10034.png',
 ' 10035.png',
 ' 10036.png',
 ' 10037.png',
 ' 10038.png',
 ' 10039.png',
 ' 1004.png',
 ' 10040.png',
 ' 10041.png',
 ' 10042.png',
 ' 10043.png',
 ' 10044.png',
 ' 10045.png',
 ' 10046.png',
 ' 10047.png',
 ' 10048.png',
 ' 10049.png',
 ' 1005.png',
 ' 10050.png',
 ' 10051.png',
 ' 10052.png',
 ' 10053.png',
 ' 10054.png',
 ' 10055.png',
 ' 10056.png',
 ' 10057.png',
 ' 10058.p

In [57]:
import pandas as pd
df = pd.DataFrame()
df['filename'] = test_filenames
df['label'] = preds
df

,filename,label
0,1.png,enface
1,10.png,enface
2,100.png,enface
3,1000.png,profile
4,10000.png,profile
...,...,...
11509,9995.png,profile
11510,9996.png,profile
11511,9997.png,enface
11512,9998.png,enface


In [58]:
for i in range(len(df)):
    if df["label"][i] == "enface":
        df["label"][i] = 0
    else: 
        df["label"][i] =1
# df.to_csv(Path('./submission.csv'), index=False)

In [59]:
df

,filename,label
0,1.png,0
1,10.png,0
2,100.png,0
3,1000.png,1
4,10000.png,1
...,...,...
11509,9995.png,1
11510,9996.png,1
11511,9997.png,0
11512,9998.png,0


In [60]:
df["filename"][0]

' 1.png'

In [61]:
for i in range(len(df)):
    df["filename"][i] = df["filename"][i].replace(" ","")
df

,filename,label
0,1.png,0
1,10.png,0
2,100.png,0
3,1000.png,1
4,10000.png,1
...,...,...
11509,9995.png,1
11510,9996.png,1
11511,9997.png,0
11512,9998.png,0


In [62]:
df["filename"][0]

'1.png'

In [63]:
df.to_csv(Path('./submission6.csv'), index=False)

In [124]:
sample_sub = pd.read_csv("sample_submission.csv")
sample_sub = sample_sub.drop("label", axis=1)
sample_sub

,filename
0,1.png
1,2.png
2,3.png
3,5.png
4,6.png
...,...
11509,11578.png
11510,11579.png
11511,11580.png
11512,11581.png


In [128]:
sample_sub['filename'][0]

'1.png'

In [126]:
pd.merge(df, sample_sub, on = "filename")

,filename,label


In [121]:
for i in range(len(sample_sub)):
    for j in range(len(sample_sub)):
        if sample_sub["filename"][i] == df["filename"][j]:
            sample_sub["label"][i] = df["label"][j]

sample_sub

,filename,label
0,1.png,1
1,2.png,1
2,3.png,1
3,5.png,1
4,6.png,1
...,...,...
11509,11578.png,1
11510,11579.png,1
11511,11580.png,1
11512,11581.png,1


In [102]:
sample_sub.to_csv(Path('./submission2.csv'), index=False)